In [1]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile

In [2]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
generate a bot that greets me
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Compiling TypeSpec...


BadRequestError: Error code: 400 - {'message': 'The provided model identifier is invalid.'}

In [ ]:
my_bot

In [ ]:
my_bot.gherkin

In [ ]:
application.generation_dir

In [ ]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc src/main.ts
!popd


In [ ]:
typespec = application._make_typespec(application_description)
typespec

In [ ]:
typescript_schema = application._make_typescript_schema(typespec.typespec_definitions)
typescript_schema

In [ ]:
drizzle = application._make_drizzle(typespec.typescript_schema)
drizzle

In [ ]:
drizzle.error_output, drizzle.reasoning, drizzle.drizzle_schema

In [ ]:
router = application._make_router(typespec.data.output.typespec_definitions)
router.score, router.data.output

In [ ]:
handlers = application._make_handlers(
    typespec.data.output.llm_functions,
    typespec.data.output.typespec_definitions,
    typescript_schema.data.output.typescript_schema,
    drizzle.data.output.drizzle_schema,
)

In [ ]:
for name, handler in handlers.items():
    print(name, handler.score, handler.depth, handler.data.output)
    if handler.score != 1:
        print(handler.data.output.feedback["stdout"])